In [ ]:
import batoid
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r_baffles.yaml")

In [ ]:
def pupil(thx, thy, nside=512, dx=0):
    scope = telescope.withGloballyShiftedOptic("M2",(dx,0,0))
    rays = batoid.RayVector.asGrid(
        optic=scope, wavelength=620e-9,
        theta_x=thx, theta_y=thy,
        nx=nside, ny=nside
    )
    rays2 = rays.copy()
    scope.stopSurface.interact(rays2)
    scope.trace(rays)
    w = ~rays.vignetted
    return rays2.x[w], rays2.y[w]

In [ ]:
def drawCircle(ax, cx, cy, r, **kwargs):
    t = np.linspace(0, 2*np.pi, 1000)
    x = r*np.cos(t)+cx
    y = r*np.sin(t)+cy
    ax.plot(x, y, **kwargs)

In [ ]:
def drawRay(ax, cx, cy, width, theta, **kwargs):
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    
    dx = np.linspace(0, 4.1, 1000)
    dy = np.ones_like(dx)*width/2

    bx = np.copy(dx)
    by = -dy
    
    dx, dy = R.dot(np.vstack([dx, dy]))
    bx, by = R.dot(np.vstack([bx, by]))
    
    dx += cx
    dy += cy
    bx += cx
    by += cy
    
    ax.plot(dx, dy, **kwargs)
    ax.plot(bx, by, **kwargs)

In [ ]:
def drawRectangle(ax, cx, cy, width, height, **kwargs):
    x = width/2*np.array([-1,-1,1,1,-1])
    y = height/2*np.array([-1,1,1,-1,-1])
    x += cx
    y += cy
    ax.plot(x, y, **kwargs)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
@interact(thx=(-2.0, 2.0, 0.01), thy=(-2.0, 2.0, 0.01), dx=(-10, 10, 1))
def modelPlot(thx=1.67, thy=0.0, dx=0.0):
    thx = np.deg2rad(thx)
    thy = np.deg2rad(thy)
    fig, ax = plt.subplots(1, 1, figsize=(24, 24))
    ax.scatter(*pupil(thx,thy,dx=dx*1e-2), s=0.1, c='k')
    ax.set_aspect('equal')
    # Primary mirror
    drawCircle(ax, 0, 0, 4.18, c='r')
    drawCircle(ax, 0, 0, 2.55, c='r')
    
    scales = {
        'M2_inner': 2.3698999752679404,
        'M2_outer': 4.502809953009087,
        'M3_inner': 1.1922312943631603,
        'M3_outer': 5.436574702296011,
#         'L1_entrance': 7.697441260764198,
#         'L1_exit': 8.106852624652701,
#         'L2_entrance': 10.748915941599885,
#         'L2_exit': 11.5564127895276,
#         'Filter_entrance': 28.082220873785978,
#         'Filter_exit': 30.91023954045243,
#         'L3_entrance': 54.67312185149621,
#         'L3_exit': 114.58705556485711
    }
    speeds = {
        'M2_inner': 16.8188788239707,
        'M2_outer': 16.8188788239707,
        'M3_inner': 53.22000661238318,
        'M3_outer': 53.22000661238318,
#         'L1_entrance': 131.76650078100135,
#         'L1_exit': 137.57031952814913,
#         'L2_entrance': 225.6949885074127,
#         'L2_exit': 237.01739037674315,
#         'Filter_entrance': 802.0137451419788,
#         'Filter_exit': 879.8810309773828,
#         'L3_entrance': 1597.8959863335774,
#         'L3_exit': 3323.60145194633
    }
    
    for k in scales:
        drawCircle(ax, -speeds[k]*thx, -speeds[k]*thy, scales[k], c='r')

#     ax.set_xlim(-50,50)
#     ax.set_ylim(-50,50)
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)

    ax.axvline(c='k')
    ax.axhline(c='k')
    fig.show()